In [1]:
import spiceypy
# Allows us to convert to ephemeris time

In [4]:
import datetime
date_today = datetime.datetime.today()

date_today = date_today.strftime("%Y-%m-%dT00:00:00")

In [5]:
print(date_today)

2024-09-28T00:00:00


In [ ]:
et_today_midnight = spiceypy